In [ ]:
##### 초기설정

from binance_f import RequestClient
from binance_f.model.constant import *
import comm.config as conf

g_api_key = conf.G_API_KEY       # binance API key
g_secret_key = conf.G_SECRET_KEY # binance secret key
coin_name = 'XRPUSDT'

request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key)

In [ ]:
##### 초기 레버리지 설정

result = request_client.change_initial_leverage(symbol=coin_name, leverage=1.0)
print(">>", result.leverage) #1.0을 리턴해야 한다.

In [ ]:
#### 호가조회

result = request_client.get_order_book(symbol = coin_name, limit = 10)
bid = result.bids[0].price #매수
ask = result.asks[0].price #매도
print(">>", bid, ask)

In [ ]:
#### 주문: LONG 지정가매수

result = request_client.post_order(
    symbol=coin_name,
    side=OrderSide.BUY,
    ordertype=OrderType.LIMIT,
    quantity = '10',
    price = '0.6655',
    timeInForce = "GTC",
    positionSide="LONG"
)
print(">>orderId:{}".format(result.orderId))

In [ ]:
##### 주문: LONG : TAKE_PROFIT
#long position을 open한 후(new 상태일 때는 take profit이 자동 close된다) take profit 주문을 넣는다.

result = request_client.post_order(
    symbol=coin_name,
    side=OrderSide.SELL,
    ordertype=OrderType.TAKE_PROFIT,
    quantity = '10',
    price = '0.6665',     #현재가 보다 더 높은 가격
    stopPrice = '0.6656', #현재가 보다 높은 가격
    positionSide="LONG"
)
print(">>orderId:{}".format(result.orderId))

In [ ]:
##### 주문: LONG POSITION : STOP_MARKET, 손실 최소화

result = request_client.post_order(
    symbol=coin_name,
    side=OrderSide.SELL,
    ordertype=OrderType.STOP_MARKET,
    stopPrice = '0.6600',   #현재가 보다 낮은 가격
    quantity = '10',
    positionSide="LONG"
)
print(">> orderId:{}".format(result.orderId))

In [ ]:
##### 주문: LONG POSITION CLOSE : MARKET

result = request_client.post_order(
    symbol=coin_name,
    side=OrderSide.SELL,
    ordertype=OrderType.MARKET,
    quantity = '10',
    positionSide="LONG"
)
print(">>orderId:{}".format(result.orderId))

In [ ]:
##### 주문: LONG POSITION CLOSE : LIMIT
result = request_client.post_order(
    symbol=coin_name,
    side=OrderSide.SELL,
    ordertype=OrderType.LIMIT,
    quantity = '10',
    price = '0.6650',
    timeInForce = "GTC",
    positionSide="LONG"
)
print(">>orderId:{} status:{}".format(result.orderId, result.status))

In [ ]:
##### 주문상태 조회

result = request_client.get_order(symbol=coin_name, orderId='15504205374')
print('>>status:',result.status) #NEW:대기, FILLED:체결, CANCELED:취소

In [ ]:
##### 주문취소(미체결 주문에 대한 취소)

result = request_client.cancel_order(symbol=coin_name, orderId='15504255448')
print('>>status:',result.status) #"CANCELED" 가 정상

In [ ]:
##### 전체 주문 조회

results = request_client.get_all_orders(symbol=coin_name)
for result in results:
    if result.status == 'NEW': #NEW:미체결, FILLED:체결
        print(">> order new:", result.status, result.price, result.type, result.positionSide,
              result.closePosition, result.executedQty, result.workingType)

In [ ]:
##### 전체 주문 취소: 미체결 주문

result = request_client.cancel_all_orders(symbol=coin_name)
print('>> result:',result.code, result.msg) #200, The operation of cancel all open order is done.

In [ ]:
##### 포지션 조회

results = request_client.get_position()
for result in results:
    if result.symbol == coin_name and result.positionSide == 'LONG':
        print('>> long position:',result.symbol, result.positionSide, result.positionAmt, result.entryPrice)
    if result.symbol == coin_name and result.positionSide == 'SHORT':
        print('>> short position:',result.symbol, result.positionSide, abs(result.positionAmt), result.entryPrice)

In [ ]:
##### 잔고조회

result = request_client.get_balance()
balance = 0
for i in result:
    balance = balance + i.balance
print('>> balance:', balance)

In [ ]:
##### 잔고조회 : 로그 없애기

result = request_client.get_balance()
balance = 0
for i in result:
    balance = balance + i.balance
print('>> balance:', balance)